In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get("https://www.reddit.com", headers=headers)
doc = BeautifulSoup(response.text, 'html.parser')

In [14]:
# test cell

# titles = doc.find_all(class_='y8HYJ-y_lTUHkQIc1mdCq')
# #for title in titles:
#     #print(title.text)
    
# links = doc.find_all(class_='s14d20sq-0', href=True)
# #for link in links:
#     #print(link['href'])
    
# upvotes = doc.find_all(class_='s1pzl9zm-1')
# #for upvote in upvotes:
#     #print(upvote.text)
    
# subreddits = doc.find_all(class_='eAfOKg')
# #for sub in subreddits:
#     #print(sub.text)

# posters = doc.find_all(class_='_2tbHP6ZydRpjI44J3syuqC')
# #for poster in posters:
#     #print(poster.text)

# comments = doc.find_all(class_='FHCV02u6Cp2zYL0fhQPsO')
# for comment in comments:
#     print(comment.text)

In [15]:
redditposts = doc.find_all(class_='_1poyrkZ7g36PawDueRza-J')

reddit_post_list = []

for post in redditposts:
    
    reddit_dict = {}
    
    title = post.find(class_='y8HYJ-y_lTUHkQIc1mdCq')
    externallink = post.find(class_='s14d20sq-0', href=True)
    thread_link = post.find(class_='SQnoC3ObvgnGjWt90zD9Z')
    upvotecount = post.find(class_='ofcg7d-1')
    poster = post.find(class_='_2tbHP6ZydRpjI44J3syuqC')
    commentcount = post.find(class_='FHCV02u6Cp2zYL0fhQPsO')
    
    if title:
        reddit_dict['Title'] = title.text
        #print(title.text)
        
    if externallink:
        reddit_dict['External URL'] = externallink['href']
    
    if thread_link:
        reddit_dict['Thread URL'] = thread_link['href']
    
    if upvotecount:
        reddit_dict['Upvote Count'] = upvotecount.text
        #print(upvotecount.text)
        
    if poster:
        reddit_dict['Poster/User'] = poster.text
        #print(poster.text)
        
    if commentcount:
        reddit_dict['Comment Count'] = commentcount.text
        #print(commentcount.text)
    reddit_post_list.append(reddit_dict)
reddit_post_list

[{'Title': 'High Five!',
  'External URL': 'https://i.imgur.com/kOsjrhX.gifv',
  'Thread URL': '/r/gifsthatkeepongiving/comments/8sdf3s/high_five/',
  'Upvote Count': '13.6k',
  'Poster/User': 'u/deathakissaway',
  'Comment Count': '267 comments'},
 {'Title': 'The U.S. has officially quit the UN Human Rights Council.',
  'External URL': 'https://thinkprogress.org/u-s-leaves-the-u-n-human-rights-council-2bf690606819/',
  'Thread URL': '/r/worldnews/comments/8sckc6/the_us_has_officially_quit_the_un_human_rights/',
  'Upvote Count': '45.2k',
  'Poster/User': 'u/AdamCannon',
  'Comment Count': '6.6k comments'},
 {'Title': "One of the most meme'd on Disney characters totally underrated.r/all",
  'Thread URL': '/r/wholesomememes/comments/8sck4b/one_of_the_most_memed_on_disney_characters/',
  'Upvote Count': '27.0k',
  'Poster/User': 'u/wffln',
  'Comment Count': '313 comments'},
 {'Title': 'Japanese fans clean up stadium after their win vs Colombia',
  'External URL': 'https://youtu.be/Yd5n-

In [16]:
df = pd.DataFrame(reddit_post_list, columns=reddit_dict.keys())
df.head()

# NOTE: not all the the rows have external URLs because some images/texts are posted directly onto reddit. 
# if there are no comments, it is an ad

,Title,External URL,Thread URL,Upvote Count,Poster/User,Comment Count
0,High Five!,https://i.imgur.com/kOsjrhX.gifv,/r/gifsthatkeepongiving/comments/8sdf3s/high_f...,13.6k,u/deathakissaway,267 comments
1,The U.S. has officially quit the UN Human Righ...,https://thinkprogress.org/u-s-leaves-the-u-n-h...,/r/worldnews/comments/8sckc6/the_us_has_offici...,45.2k,u/AdamCannon,6.6k comments
2,One of the most meme'd on Disney characters to...,NaN,/r/wholesomememes/comments/8sck4b/one_of_the_m...,27.0k,u/wffln,313 comments
3,Japanese fans clean up stadium after their win...,https://youtu.be/Yd5n--lfdqg,/r/videos/comments/8sck9f/japanese_fans_clean_...,23.1k,u/wrdb2007,1.2k comments
4,Every Sailor has a story. As an Aviation Rescu...,https://alb.reddit.com/c?q=CgADBTEoKIfAlSMKAAU...,/comments/8redl4/every_sailor_has_a_story_as_a...,0,u/AmericasNavy,comment


In [17]:
df.to_csv("reddit_homepage.csv", index=False)

In [18]:
right_now = datetime.datetime.now()
#right_now.strftime("%Y-%m-%d_%H_%M_%S")

In [19]:
filename = "reddit-homepage-" + right_now.strftime("%Y-%m-%d_%-I%p") + ".csv"
df.to_csv(filename, index=False)

In [21]:
htmldf = df.to_html()

In [23]:
response2 = requests.post(
        "https://api.mailgun.net/v3/sandbox6828290367e84edd8948e4975083acfd.mailgun.org/messages",
        auth=("api", "API-KEY"),
        files=[("attachment", (filename, open(filename,"rb").read()))],
        data={"from": "Reddit Report <amw2262@columbia.edu>",
              "to": ["amw2262@columbia.edu"],
              "subject": "Here is your" + " "+ datetime.datetime.now().strftime("%-I%p") + " " + "briefing",
              "text": "Here you go.",
              "html": htmldf})
response2.text

'{\n  "id": "<20180620020235.1.1A0850A15AA1D8C1@sandbox6828290367e84edd8948e4975083acfd.mailgun.org>",\n  "message": "Queued. Thank you."\n}'